In [ ]:
# Step 1: Install PostgreSQL and psycopg2 for PostgreSQL connection in Colab
!apt-get -y install postgresql postgresql-contrib
!pip install psycopg2-binary

In [2]:
# Step 2: Start PostgreSQL service
import os
os.system('service postgresql start')

0

In [3]:
# Step 2.5: Change the password for the postgres user to 'postgres'
os.system('sudo -u postgres psql -c "ALTER USER postgres PASSWORD \'postgres\';"')

0

In [4]:
# Step 3: Set up PostgreSQL and create a new database
import psycopg2

# Connect to the default PostgreSQL database
conn = psycopg2.connect(database="postgres", user="postgres", password="postgres", host="localhost", port="5432")
conn.autocommit = True
cur = conn.cursor()

# Create a new database
cur.execute("DROP DATABASE IF EXISTS sales_data_db;")
cur.execute("CREATE DATABASE sales_data_db;")
cur.close()
conn.close()

# Connect to the new database
conn = psycopg2.connect(database="sales_data_db", user="postgres", password="postgres", host="localhost", port="5432")
cur = conn.cursor()

In [5]:
# Step 4: Download and load the sales/advertising dataset
!wget https://raw.githubusercontent.com/selva86/datasets/master/Advertising.csv -O advertising.csv

import pandas as pd

# Load the CSV into a DataFrame
df = pd.read_csv('advertising.csv')
print("Sample data:")
print(df.head())

# Create a table and insert data into PostgreSQL
cur.execute("""
    CREATE TABLE sales_data (
        TV FLOAT,
        Radio FLOAT,
        Newspaper FLOAT,
        Sales FLOAT
    );
""")

--2024-09-20 19:20:58--  https://raw.githubusercontent.com/selva86/datasets/master/Advertising.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4756 (4.6K) [text/plain]
Saving to: ‘advertising.csv’

advertising.csv     100%[===================>]   4.64K  --.-KB/s    in 0s      

2024-09-20 19:20:58 (48.4 MB/s) - ‘advertising.csv’ saved [4756/4756]

Sample data:
   Unnamed: 0     TV  radio  newspaper  sales
0           1  230.1   37.8       69.2   22.1
1           2   44.5   39.3       45.1   10.4
2           3   17.2   45.9       69.3    9.3
3           4  151.5   41.3       58.5   18.5
4           5  180.8   10.8       58.4   12.9


In [6]:
# Insert data from the DataFrame into the PostgreSQL table
for _, row in df.iterrows():
    cur.execute(
        "INSERT INTO sales_data (TV, Radio, Newspaper, Sales) VALUES (%s, %s, %s, %s)",
        (row['TV'], row['radio'], row['newspaper'], row['sales'])
    )

conn.commit()
print("Data successfully inserted into PostgreSQL.")

Data successfully inserted into PostgreSQL.


In [7]:
# Add a variable that only goes up by tens
cur.execute("""ALTER TABLE sales_data ADD COLUMN TV_Rounded FLOAT;
UPDATE sales_data
SET TV_Rounded = ROUND(TV / 10.0) * 10;"""
)

Demonstrate common SQL commands

In [8]:
def fetch_and_display(cur, query):
    """
    Execute a SQL query and display as a pandas DataFrame.
    """
    try:
        cur.execute(query)
        rows = cur.fetchall()
        col_names = [desc[0] for desc in cur.description]
        df_results = pd.DataFrame(rows, columns=col_names)
        #print(df_results)
        return df_results
    except Exception as e:
        print(f"An error occurred: {e}")

In [9]:
# 1. SELECT command
print("SELECT command example:")
fetch_and_display(cur,"SELECT * FROM sales_data LIMIT 5;")

SELECT command example:


,tv,radio,newspaper,sales,tv_rounded
0,230.1,37.8,69.2,22.1,230.0
1,44.5,39.3,45.1,10.4,40.0
2,17.2,45.9,69.3,9.3,20.0
3,151.5,41.3,58.5,18.5,150.0
4,180.8,10.8,58.4,12.9,180.0


In [10]:
# 2. WHERE clause
print("WHERE clause example:")
fetch_and_display(cur,"SELECT * FROM sales_data WHERE Sales > 15;")

WHERE clause example:


,tv,radio,newspaper,sales,tv_rounded
0,230.1,37.8,69.2,22.1,230.0
1,151.5,41.3,58.5,18.5,150.0
2,214.7,24.0,4.0,17.4,210.0
3,204.1,32.9,46.0,19.0,200.0
4,195.4,47.7,52.9,22.4,200.0
...,...,...,...,...,...
70,191.1,28.7,18.2,17.3,190.0
71,286.0,13.9,3.7,15.9,290.0
72,166.8,42.0,3.6,19.6,170.0
73,149.7,35.6,6.0,17.3,150.0


In [11]:
# 3. ORDER BY clause
print("ORDER BY clause example:")
fetch_and_display(cur,"SELECT * FROM sales_data ORDER BY Sales DESC;")

ORDER BY clause example:


,tv,radio,newspaper,sales,tv_rounded
0,276.9,48.9,41.8,27.0,280.0
1,287.6,43.0,71.8,26.2,290.0
2,283.6,42.0,66.2,25.5,280.0
3,289.7,42.3,51.2,25.4,290.0
4,243.2,49.0,44.3,25.4,240.0
...,...,...,...,...,...
195,13.1,0.4,25.6,5.3,10.0
196,5.4,29.9,9.4,5.3,10.0
197,8.6,2.1,1.0,4.8,10.0
198,4.1,11.6,5.7,3.2,0.0


In [13]:
# 4. GROUP BY clause
print("GROUP BY clause example:")
fetch_and_display(cur,"SELECT TV_Rounded, COUNT(*) FROM sales_data GROUP BY TV_Rounded LIMIT 5;")

GROUP BY clause example:


,tv_rounded,count
0,0.0,2
1,70.0,9
2,80.0,8
3,100.0,7
4,150.0,4


In [14]:
# 5. JOIN example - Self join (TODO: replace with join on another dataset)
print("Self join example:")
fetch_and_display(cur,"""
    SELECT a.TV, b.Radio
    FROM sales_data a
    JOIN sales_data b
    ON a.TV = b.TV
    LIMIT 5;
""")

Self join example:


,tv,radio
0,230.1,37.8
1,44.5,39.3
2,17.2,45.9
3,17.2,4.1
4,151.5,41.3


In [20]:
# 6. INSERT INTO
print("INSERT INTO example:")
cur.execute("INSERT INTO sales_data (TV, Radio, Newspaper, Sales, TV_Rounded) VALUES (300, 50, 100, 25, 300);")
conn.commit()

INSERT INTO example:


In [21]:
# 7. UPDATE
print("UPDATE example:")
cur.execute("UPDATE sales_data SET Sales = 30 WHERE TV = 300;")
conn.commit()

UPDATE example:


In [22]:
# 8. DELETE
print("DELETE example:")
cur.execute("DELETE FROM sales_data WHERE TV = 300;")
conn.commit()

DELETE example:


In [23]:
# 9. CREATE TABLE
print("CREATE TABLE example:")
cur.execute("""
    CREATE TABLE new_sales_data (
        TV FLOAT,
        Radio FLOAT,
        Newspaper FLOAT,
        Sales FLOAT
    );""")

CREATE TABLE example:


In [24]:
# 10. DROP TABLE
print("DROP TABLE example:")
cur.execute("DROP TABLE IF EXISTS new_sales_data;")

DROP TABLE example:


In [25]:
# 11. CASE WHEN
print("CASE WHEN example:")
fetch_and_display(cur,"""
    SELECT TV,
           CASE WHEN Sales > 15 THEN 'High'
                WHEN Sales > 10 THEN 'Medium'
                ELSE 'Low'
           END as Sales_Category
    FROM sales_data;
""")

CASE WHEN example:


,tv,sales_category
0,230.1,High
1,44.5,Medium
2,17.2,Low
3,151.5,High
4,180.8,Medium
...,...,...
195,38.2,Low
196,94.2,Low
197,177.0,Medium
198,283.6,High


In [28]:
# 12. DISTINCT
print("DISTINCT example:")
fetch_and_display(cur,"SELECT DISTINCT TV_Rounded FROM sales_data LIMIT 5;")

DISTINCT example:


,tv_rounded
0,0.0
1,190.0
2,130.0
3,160.0
4,90.0


In [29]:
# 13. LEFT JOIN
# Creating another table for demonstration
cur.execute("""
    CREATE TABLE ad_channels (
        TV FLOAT,
        Channel VARCHAR(50)
    );
""")
cur.execute("INSERT INTO ad_channels (TV, Channel) VALUES (200, 'Sports'), (150, 'News'), (230, 'Music');")

In [31]:
# LEFT JOIN
print("LEFT JOIN example:")
fetch_and_display(cur,"""
    SELECT a.TV_Rounded, b.Channel
    FROM sales_data a
    LEFT JOIN ad_channels b
    ON a.TV_Rounded = b.TV;
""")

LEFT JOIN example:


,tv_rounded,channel
0,200.0,Sports
1,200.0,Sports
2,200.0,Sports
3,200.0,Sports
4,200.0,Sports
...,...,...
195,100.0,None
196,100.0,None
197,100.0,None
198,100.0,None


In [32]:
# 14. UNION
print("UNION example:")
fetch_and_display(cur,"""
    SELECT TV FROM sales_data
    UNION
    SELECT TV FROM ad_channels;
""")

UNION example:


,tv
0,136.2
1,18.7
2,7.8
3,224.0
4,205.0
...,...
188,209.6
189,31.5
190,243.2
191,163.3


In [33]:
# 15. SUM and COUNT
print("SUM and COUNT example:")
fetch_and_display(cur,"SELECT SUM(Sales), COUNT(*) FROM sales_data;")

SUM and COUNT example:


,sum,count
0,2804.5,200


In [35]:
# 16. Date-time manipulation (demonstration with date functions)
print("Current date-time:")
fetch_and_display(cur,"SELECT NOW();")

Current date-time:


,now
0,2024-09-20 19:24:12.162211+00:00


In [36]:
# 17. String formatting and substring
print("String formatting example:")
fetch_and_display(cur,"SELECT SUBSTRING('AdvertisingSales', 1, 10);")

String formatting example:


,substring
0,Advertisin


In [37]:
# 18. Window function - RANK()
print("Window function - RANK example:")
fetch_and_display(cur,"""
    SELECT TV, Sales, RANK() OVER (ORDER BY Sales DESC) as sales_rank
    FROM sales_data;
""")

Window function - RANK example:


,tv,sales,sales_rank
0,276.9,27.0,1
1,287.6,26.2,2
2,283.6,25.5,3
3,289.7,25.4,4
4,243.2,25.4,4
...,...,...,...
195,13.1,5.3,196
196,5.4,5.3,196
197,8.6,4.8,198
198,4.1,3.2,199


In [38]:
# 19. Subqueries
print("Subquery example:")
fetch_and_display(cur,"""
    SELECT * FROM sales_data
    WHERE Sales = (SELECT MAX(Sales) FROM sales_data);
""")

Subquery example:


,tv,radio,newspaper,sales,tv_rounded
0,276.9,48.9,41.8,27.0,280.0


In [41]:
# 20. HAVING vs WHERE
print("HAVING vs WHERE example:")
fetch_and_display(cur,"""
    SELECT TV_Rounded, SUM(Sales) as total_sales
    FROM sales_data
    GROUP BY TV_Rounded
    HAVING SUM(Sales) > 15
    LIMIT 5;
""")

HAVING vs WHERE example:


,tv_rounded,total_sales
0,190.0,96.9
1,130.0,69.0
2,160.0,59.9
3,90.0,81.1
4,20.0,76.2


In [43]:
# 21. LAG and LEAD
print("LAG and LEAD example:")
fetch_and_display(cur,"""
    SELECT TV, Sales,
           LAG(Sales) OVER (ORDER BY TV) as previous_sales,
           LEAD(Sales) OVER (ORDER BY TV) as next_sales
    FROM sales_data;
""")

LAG and LEAD example:


,tv,sales,previous_sales,next_sales
0,0.7,1.6,NaN,3.2
1,4.1,3.2,1.6,5.3
2,5.4,5.3,3.2,5.5
3,7.3,5.5,5.3,6.6
4,7.8,6.6,5.5,5.7
...,...,...,...,...
195,289.7,25.4,26.2,12.8
196,290.7,12.8,25.4,21.4
197,292.9,21.4,12.8,20.7
198,293.6,20.7,21.4,23.8


In [44]:
# 22. Indexing (Basic Example)
print("Index created on 'TV' column.")
cur.execute("CREATE INDEX idx_tv ON sales_data (TV);")

Index created on 'TV' column.


In [45]:
# 23. Running totals
print("Running total example:")
fetch_and_display(cur,"""
    SELECT TV, Sales,
           SUM(Sales) OVER (ORDER BY TV ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as running_total
    FROM sales_data;
""")

Running total example:


,tv,sales,running_total
0,0.7,1.6,1.6
1,4.1,3.2,4.8
2,5.4,5.3,10.1
3,7.3,5.5,15.6
4,7.8,6.6,22.2
...,...,...,...
195,289.7,25.4,2725.8
196,290.7,12.8,2738.6
197,292.9,21.4,2760.0
198,293.6,20.7,2780.7


In [46]:
# 24. MAX and MIN
print("MAX and MIN example:")
fetch_and_display(cur,"SELECT MAX(Sales), MIN(Sales) FROM sales_data;")

MAX and MIN example:


,max,min
0,27.0,1.6


In [47]:
# Close the database connection
cur.close()
conn.close()